In [44]:
import matplotlib.pyplot as plt 
#import scipy.misc
import numpy as np
import json
from scipy import misc

from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, Cropping2D, Concatenate

from keras.layers import MaxPooling2D, ZeroPadding2D, BatchNormalization
from keras.models import Model
from keras.utils.layer_utils import print_summary
from keras.preprocessing.image import ImageDataGenerator

from keras import __version__
print(__version__)

2.0.5


In [ ]:
train_folder = '/home/ubuntu/data/sar/experiment_crops/exp/train'
valid_folder = '/home/ubuntu/data/sar/experiment_crops/exp/valid'


In [57]:
main_shape = (140, 140 ,1)
aux_shape = (1, 1, 1)
main_input = Input(shape=main_shape, name='main_input')

aux_input = Input(shape=aux_shape, name='aux_input')
x2 = Dense(1, activation='relu')(aux_input)

num_classes = 3

x = Cropping2D(cropping=((10, 10), (10, 10)))(main_input)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Flatten()(x)

added = keras.layers.Add()([x, x2])
out = Dense(num_classes, activation='softmax')(added)


model = keras.models.Model(inputs=[main_input, aux_input], outputs=out)


In [58]:
model.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
main_input (InputLayer)          (None, 140, 140, 1)   0                                            
____________________________________________________________________________________________________
cropping2d_6 (Cropping2D)        (None, 120, 120, 1)   0           main_input[0][0]                 
____________________________________________________________________________________________________
conv2d_20 (Conv2D)               (None, 120, 120, 32)  320         cropping2d_6[0][0]               
____________________________________________________________________________________________________
max_pooling2d_9 (MaxPooling2D)   (None, 60, 60, 32)    0           conv2d_20[0][0]                  
___________________________________________________________________________________________

In [56]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [99]:
import json

with open('/home/ubuntu/data/sar/experiment_crops/train/oil.json') as json_data:
    oil_json = json.load(json_data)
    json_data.close()
    
with open('/home/ubuntu/data/sar/experiment_crops/train/turbine.json') as json_data:
    turbine_json = json.load(json_data)
    json_data.close()
    
with open('/home/ubuntu/data/sar/experiment_crops/train/other.json') as json_data:
    other_json = json.load(json_data)
    json_data.close()
    
all_json = list(other_json.items()) + list(oil_json.items()) + list(turbine_json.items())

In [ ]:

# I think I found the bit that Andrew did.. not sure if we should do it that way but the links are here:    
""" Based heavily on:
https://shaoanlu.wordpress.com/2017/04/10/a-simple-pseudo-labeling-function-implementation-in-keras/
https://github.com/shaoanlu/dogs-vs-cats-redux/blob/master/res50_incepV3_Xcept.ipynb
https://github.com/asmith26/learningWithKaggle/blob/master/as/the-nature-conservancy-fisheries-monitoring_v1-keras2-pseudo_labelling.ipynb
"""

# I would try doing the .fit which is all the data as you suggested, you need to make sure the data is ordered correctly, 
# then pass it into the following. 
# I think this needs to have the shapes ([(num_images, 140, 140, 1), (num_images, 1)], (num_images, 1))
model.fit([img, features], label)         